In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Initialize Solar Details
capacity_kWp = 88
specific_yield = 1216.94  # in kWh/kWp/year
performance_drop = 0.01  # 1% drop annually
years_projection = 25

# Calculate PV Generation for each year
pv_generation_rates = []
pv_generations = []

for year in range(1, years_projection + 1):
    # Calculate PV Power Generation Rate
    pv_generation_rate = specific_yield * ((1 - performance_drop) ** (year - 1))
    pv_generation_rates.append(pv_generation_rate)

    # Calculate PV Power Generation for this year
    pv_generation = capacity_kWp * pv_generation_rate
    pv_generations.append(pv_generation)

# Output PV Generation Rates and Generations
for year in range(years_projection):
    print(f"Year {year + 1}: PV Generation Rate = {pv_generation_rates[year]:.2f} kWh/kWp/year, PV Generation = {pv_generations[year]:.2f} kWh/year")


In [ ]:
# Building details
baseline_consumption = 94000  # in kWh/year
annual_hike = 0.02  # 2% annual hike
annual_reduction = 0.03  # 3% annual reduction

# Financial parameters
electricity_tariff = 0.298  # RM/kWh
tnb_buyback_rate = 0.25  # RM/kWh

# Lists to store the results
annual_consumptions = []
consumed_pv_powers = []
excess_energy_exported = []
energy_savings = []
exported_energy_savings = []

for year in range(1, years_projection + 1):
    # Calculate annual consumption with hike or reduction applied
    if year == 1:
        annual_consumption = baseline_consumption
    else:
        # Assuming annual reduction here (it can be replaced with hike logic as needed)
        annual_consumption = annual_consumptions[-1] * (1 - annual_reduction)
    
    annual_consumptions.append(annual_consumption)
    
    # Calculate Consumed PV Power and Excess Energy Exported
    pv_generation = pv_generations[year - 1]
    consumed_pv_power = min(pv_generation, annual_consumption)
    excess_energy = max(0, pv_generation - annual_consumption)
    
    consumed_pv_powers.append(consumed_pv_power)
    excess_energy_exported.append(excess_energy)
    
    # Calculate Energy Savings
    consumption_saving = consumed_pv_power * electricity_tariff
    export_saving = excess_energy * tnb_buyback_rate
    
    energy_savings.append(consumption_saving)
    exported_energy_savings.append(export_saving)

# Output the results
for year in range(years_projection):
    print(f"Year {year + 1}: Consumption = {annual_consumptions[year]:.2f} kWh, "
          f"Consumed PV = {consumed_pv_powers[year]:.2f} kWh, "
          f"Excess Exported = {excess_energy_exported[year]:.2f} kWh, "
          f"Energy Saving = RM{energy_savings[year]:.2f}, "
          f"Export Saving = RM{exported_energy_savings[year]:.2f}")


In [ ]:
import numpy as np

# Initial Investment (Negative as it's a cost)
initial_investment = - (capacity_kWp * 6000 + 0)  # Total Cost for Year 0

# Cash Flows: List including initial investment and subsequent yearly cash flows
cash_flows = [initial_investment] + [energy_savings[year] + exported_energy_savings[year] - opex for year in range(years_projection)]

# Calculate IRR
irr = np.irr(cash_flows)
print(f"IRR: {irr * 100:.2f}%")


In [ ]:
# Calculate Cumulative Cash Flow to determine payback year
cumulative_savings = initial_investment
payback_period = None

for year in range(years_projection):
    annual_cash_flow = energy_savings[year] + exported_energy_savings[year] - opex
    cumulative_savings += annual_cash_flow
    
    if cumulative_savings >= 0 and payback_period is None:
        payback_period = year + 1  # Payback period year
        break

if payback_period:
    print(f"Payback Period: {payback_period} years")
else:
    print("Payback Period: Not achieved within the projection period")


In [ ]:
# Calculate Solar Consumption Percentage for each year
solar_consumption_percentages = []

for year in range(years_projection):
    consumption_percentage = (consumed_pv_powers[year] / annual_consumptions[year]) * 100
    solar_consumption_percentages.append(consumption_percentage)

# Calculate average consumption percentage
average_solar_consumption_percentage = sum(solar_consumption_percentages) / years_projection
print(f"Average Solar Consumption Percentage: {average_solar_consumption_percentage:.2f}%")


Full Code Markdown Below

In [11]:
pip install scipy


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 217.9 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 220.2 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 220.2 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 220.2 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 220.2 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 220.2 kB/s eta 0:00:01
     -------------------------- ------------ 41.0/60.8 kB 93.7 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.8 kB 114.1 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 124.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 MB ? eta -:


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd
import scipy.optimize as opt

# Solar Details
capacity_kWp = 88
specific_yield = 1216.94  # in kWh/kWp/year
performance_drop = 0.01  # 1% drop annually
years_projection = 25

# Building Details
baseline_consumption = 94000  # in kWh/year
annual_reduction = 0.03  # 3% annual reduction

# Financial Parameters
electricity_tariff = 0.298  # RM/kWh
tnb_buyback_rate = 0.25  # RM/kWh
cost_per_kwp = 6000  # RM/kWp
additional_structure_cost = 0  # RM
opex = 0  # Operating expenses assumed to be zero for simplicity

# Initialize lists to store the results
pv_generation_rates = []
pv_generations = []
annual_consumptions = []
consumed_pv_powers = []
excess_energy_exported = []
energy_savings = []
exported_energy_savings = []
solar_consumption_percentages = []
cumulative_cash_flows = []

# Calculate PV Generation and Annual Consumption for each year
cumulative_savings = - (capacity_kWp * cost_per_kwp + additional_structure_cost)
initial_investment = cumulative_savings

for year in range(1, years_projection + 1):
    # PV Power Generation Rate Calculation
    pv_generation_rate = specific_yield * ((1 - performance_drop) ** (year - 1))
    pv_generation_rates.append(pv_generation_rate)

    # PV Power Generation (kWh/year)
    pv_generation = capacity_kWp * pv_generation_rate
    pv_generations.append(pv_generation)

    # Annual Consumption Calculation
    if year == 1:
        annual_consumption = baseline_consumption
    else:
        annual_consumption = annual_consumptions[-1] * (1 - annual_reduction)
    annual_consumptions.append(annual_consumption)

    # Consumed PV Power and Excess Energy Exported
    consumed_pv_power = min(pv_generation, annual_consumption)
    excess_energy = max(0, pv_generation - annual_consumption)
    consumed_pv_powers.append(consumed_pv_power)
    excess_energy_exported.append(excess_energy)

    # Energy Savings Calculations
    consumption_saving = consumed_pv_power * electricity_tariff
    export_saving = excess_energy * tnb_buyback_rate
    energy_savings.append(consumption_saving)
    exported_energy_savings.append(export_saving)

    # Solar Consumption Percentage
    consumption_percentage = (consumed_pv_power / annual_consumption) * 100
    solar_consumption_percentages.append(consumption_percentage)

    # Cumulative Cash Flow Calculation
    annual_cash_flow = consumption_saving + export_saving - opex
    cumulative_savings += annual_cash_flow
    cumulative_cash_flows.append(cumulative_savings)




In [13]:
# Create a DataFrame to tabulate the results
data = {
    'Year': list(range(1, years_projection + 1)),
    'PV Generation Rate (kWh/kWp/year)': pv_generation_rates,
    'PV Generation (kWh/year)': pv_generations,
    'Annual Consumption (kWh/year)': annual_consumptions,
    'Consumed PV Power (kWh/year)': consumed_pv_powers,
    'Excess Energy Exported (kWh/year)': excess_energy_exported,
    'Energy Consumption Saving (RM)': energy_savings,
    'Exported Energy Saving (RM)': exported_energy_savings,
    'Cumulative Cash Flow (RM)': cumulative_cash_flows,
    'Solar Consumption Percentage (%)': solar_consumption_percentages
}

results_df = pd.DataFrame(data)

# Export the DataFrame to an Excel file
results_df.to_excel('solar_financial_model_results.xlsx', index=False)

# Calculate IRR
cash_flows = [initial_investment] + [energy_savings[year] + exported_energy_savings[year] - opex for year in range(years_projection)]
irr = opt.newton(lambda r: np.npv(r, cash_flows), 0.1)
print(f"IRR: {irr * 100:.2f}%")

# Determine Payback Period
payback_period = None
for year, cumulative_cash_flow in enumerate(cumulative_cash_flows, start=1):
    if cumulative_cash_flow >= 0:
        payback_period = year
        break

if payback_period:
    print(f"Payback Period: {payback_period} years")
else:
    print("Payback Period: Not achieved within the projection period")

# Calculate average solar consumption percentage
average_solar_consumption_percentage = sum(solar_consumption_percentages) / years_projection
print(f"Average Solar Consumption Percentage: {average_solar_consumption_percentage:.2f}%")

AttributeError: module 'numpy' has no attribute 'npv'